# Link data objects to sample-level metadata

We already saw how to link data objects to entities representing features during ingestion.

For sample-level metadata, the underlying schema is often more complicated, and hence it's best done in a separate step.

Here, we walk through this process.

In [ ]:
import lamindb as ln
import lamindb.schema as lns
import lamindb.knowledge as lnk

ln.nb.header()

Samples, i.e., metadata associated with observations, are linked with the same approach post-ingestion.

Let's first query an scRNA-seq dataset stored as an `.h5ad` file.

In [ ]:
dobject = ln.select(lns.DObject, suffix=".h5ad").first()

In [ ]:
dobject

For instance, let's annotate a scRNA-seq dataset with its readout type (scRNA-seq), the tissue, and the species.

## Species

In [ ]:
species = lns.bionty.Species(common_name="mouse")

In [ ]:
species

## Tissue

In [ ]:
tissue_lookup = lnk.Tissue().lookup

In [ ]:
tissue_lookup.afferent_lymphatic_vessel

In [ ]:
tissue = lns.bionty.Tissue(id=tissue_lookup.afferent_lymphatic_vessel)

In [ ]:
tissue

## Readout

In [ ]:
lnk.lookup.readout.single_cell_RNA_sequencing

In [ ]:
readout = lns.wetlab.Readout(efo_id=lnk.lookup.readout.single_cell_RNA_sequencing)

In [ ]:
readout

## Create a biosample record

In [ ]:
biosample = lns.wetlab.Biosample(name="Mouse Lymph Node")

In [ ]:
biosample.tissue = tissue

In [ ]:
biosample.species_id = species.id  # need to have a besser UX around this

Currently, we still work with another entity `biometa`. We'll like drop it.

In [ ]:
biometa = lns.wetlab.Biometa(name="Mouse Lymph Node scRNA-seq")

In [ ]:
biometa.readout = readout

In [ ]:
biometa.biosample = biosample

Link against the data object:

In [ ]:
biometa.dobjects.append(dobject)

## Add to the DB

We can add everything to the DB in one transaction:

In [ ]:
ln.add(biometa)

## Query for linked metadata

In [ ]:
stmt = ln.select(lns.wetlab.Biometa).join(
    lns.wetlab.Readout, efo_id=lnk.lookup.readout.single_cell_RNA_sequencing
)
stmt.df()

In [ ]:
ln.select(lns.DObject).join(lns.DObject.biometas).where(
    lns.wetlab.Biometa.id == stmt.first().id
).df()